In [ ]:
import torch
import multiprocessing
import os

# Function to set CPU affinity for a specific process
def set_process_affinity(core_id):
    pid = os.getpid()  # Get the current process ID
    os.sched_setaffinity(pid, {core_id})  # Bind the process to specific core(s)

# Function to perform PyTorch operations pinned to a specific CPU core
def matrix_multiplication_shared_memory(core_id, shared_tensor, shape):
    set_process_affinity(core_id)
    print(f"Process {os.getpid()} running on CPU core {core_id}")

    # Perform a matrix multiplication operation
    x = torch.rand(shape)
    y = torch.mm(x, x)  # Matrix multiplication

    # Copy result to shared memory
    with shared_tensor.get_lock():  # Synchronize access
        shared_tensor_arr = torch.frombuffer(shared_tensor.get_obj(), dtype=torch.float32)
        shared_tensor_arr[:] = y.flatten()  # Copy data to shared memory

if __name__ == '__main__':
    shape = (1000, 1000)
    tensor_size = shape[0] * shape[1]

    # Create a shared memory array for inter-process communication
    shared_tensor = multiprocessing.Array('f', tensor_size)  # Shared memory of float32 type

    # Create two separate processes assigned to different CPU cores
    process_1 = multiprocessing.Process(target=matrix_multiplication_shared_memory, args=(0, shared_tensor, shape))
    process_2 = multiprocessing.Process(target=matrix_multiplication_shared_memory, args=(1, shared_tensor, shape))

    # Start the processes
    process_1.start()
    process_2.start()

    # Wait for processes to complete
    process_1.join()
    process_2.join()

    # Retrieve results from shared memory and reshape
    result_tensor = torch.frombuffer(shared_tensor.get_obj(), dtype=torch.float32).reshape(shape)
    print(f"Result tensor from shared memory:\n {result_tensor}")
